In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkt



airport_df = pd.read_excel("C:/Users/longl/airports_processed.xlsx") 

airport_df = airport_df.dropna(subset=['latitude_deg', 'longitude_deg'])

# Convert airport DataFrame to GeoDataFrame
airport_gdf = gpd.GeoDataFrame(
    airport_df,
    geometry=gpd.points_from_xy(airport_df['longitude_deg'], airport_df['latitude_deg']),
    crs="EPSG:4326"
)

print(airport_gdf)

# Read CSV (wildfire data)
wildfire_df = pd.read_csv("C:/Users/longl/BurnData.csv")  # Use actual filename

# Convert wildfire DataFrame to GeoDataFrame
wildfire_gdf = gpd.GeoDataFrame(
    wildfire_df,
    geometry=wildfire_df['geometry'].apply(wkt.loads),  # If geometry is in WKT format
    crs="EPSG:4326"
)

# 3. Convert wildfire polygons to centroids (for point-to-point distance)
wildfire_gdf['centroid'] = wildfire_gdf.geometry.centroid
wildfire_centroids = gpd.GeoDataFrame(wildfire_gdf[['unique_id']], geometry=wildfire_gdf['centroid'], crs="EPSG:4326")

# 4. Reproject both GeoDataFrames to metric CRS (meters) for distance calc
airport_gdf = airport_gdf.to_crs(epsg=3857)
wildfire_centroids = wildfire_centroids.to_crs(epsg=3857)

# 5. Compute nearest airport using spatial join
nearest = gpd.sjoin_nearest(
    wildfire_centroids,
    airport_gdf[['name', 'geometry']],
    how='left',
    distance_col='distance_m'
)


result = nearest[['unique_id', 'name', 'distance_m']]
# Rename Columns
result = result.rename(columns={
    'unique_id': 'Fire ID',
    'name': 'CLOSEST_AIRPORT_NAME',
    'distance_m': 'DISTANCE'
})
print(result)

     ident iata_code icao_code local_code  runway_lengths  \
0      0R7       NaN       NaN        0R7    5.000000e+03   
1      5K2       NaN       NaN        5K2    5.000000e+03   
2      C56       NaN       NaN        C56    5.001000e+03   
3      D38       IUA       NaN        IUA    5.500000e+03   
4      FWB       NaN       NaN        FWB    5.000000e+03   
...    ...       ...       ...        ...             ...   
1389  PHMU       MUE      PHMU        MUE    5.197000e+03   
1390  PHNL       HNL      PHNL        HNL    6.952900e+17   
1391  PHNY       LNY      PHNY        LNY    5.001000e+03   
1392  PHOG       OGG      PHOG        OGG    6.995000e+03   
1393  PHTO       ITO      PHTO        ITO    5.600980e+07   

                                       name            type  \
0                     The Red River Airport   small_airport   
1                 Tribune Municipal Airport   small_airport   
2                                Bult Field   small_airport   
3              

C:\Users\longl\AppData\Local\Temp\ipykernel_29064\326016070.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  wildfire_gdf['centroid'] = wildfire_gdf.geometry.centroid


            Fire ID               CLOSEST_AIRPORT_NAME      DISTANCE
0        2020_76998  The Florida Keys Marathon Airport  5.743588e+06
1        2020_76999  The Florida Keys Marathon Airport  5.744652e+06
2        2020_77025  The Florida Keys Marathon Airport  5.560842e+06
3        2020_77014  The Florida Keys Marathon Airport  5.855872e+06
4        2020_77029  The Florida Keys Marathon Airport  5.682574e+06
...             ...                                ...           ...
492744   2024_87341      Houlton International Airport  1.848502e+07
492745   2024_87351      Houlton International Airport  1.847418e+07
492746   2024_87361      Houlton International Airport  1.848727e+07
492747  2024_105694      Houlton International Airport  1.981625e+07
492748  2024_105714      Houlton International Airport  1.986292e+07

[492749 rows x 3 columns]
